In [ ]:

# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Agent - Orchestration with LLM 2

## Set configuration

In [1]:
# @title Package install
%pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

In [3]:
# @title Define constants

PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

In [5]:
# @title Import packages

import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding

In [6]:
# @title Initialize Vertex AI

# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

In [8]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을  참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 영화추천 : http://media.com/영화추천
  2. 음악추천 : http://media.com/음악추천
  3. 기타 : http://media.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def agent_1(chat_history, query):

  prompt =f"""
          당신은 영화정보를 제공하는AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 영화 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 영화를 선정 하는 이유를 설명 해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}
  """
  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],

  )

  return responses.text

def agent_2(chat_history, query):

  prompt =f"""
          당신은 음악정보를 제공하는 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 오늘 날씨에 맞는 음악을 추천해주세요.
          2. 추천하는 이유를 자세히 설명해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_3(chat_history, query):

  prompt =f"""
          당신은 일상적인 대화가 가능한 AI Agent 입니다.
          가벼운 대화를 진행하되 혹시 질문이 있을 경우 해당 질문에 대해서 답변해주세요.
          전체 답변은 10줄을 넘지 않게 해주세요.
          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_orchestration(chat_history, query):

  classified = query_classification(chat_history, query)
  task = eval(classified)['classification']['class']

  print(f"Agent Task : {task}")

  if task == '영화추천':
    return agent_1(chat_history, query)
  elif task == '음악추천':
    return agent_2(chat_history, query)
  else:
    return agent_3(chat_history, query)


In [ ]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  response = agent_orchestration(chat_history, query)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 오늘 날씨 검색해서 그에 맞는 음악 추천해주세요.
Agent Task : 음악추천


AI Agent : 오늘(12월 27일) 날씨에 맞는 음악 추천입니다.  제공된 정보에 따르면, 서울의 날씨는 오전에 맑음에서 오후에 구름많음으로 변화하고, 강수확률은 오후에 30%입니다.  기온은 최저 -2도에서 최고 1도로 예상됩니다.  로스앤젤레스의 날씨는 대체로 맑음이며, 기온은 최저 50°F에서 최고 65°F입니다.

따라서, 오늘 날씨에 맞는 음악 추천은 다음과 같습니다.

1. **서울:** 오후에 쌀쌀하고 구름이 낄 가능성이 있으므로, 차분하고 따뜻한 느낌의 재즈나 팝 발라드를 추천합니다.  예를 들어, 노라 존스, 혹은 어쿠스틱 기타 연주곡이 좋습니다.  강수확률을 고려하여, 잔잔한 비오는 날의 감성을 담은 음악도 좋습니다.

2. **로스앤젤레스:** 맑고 따뜻한 날씨이므로, 경쾌하고 밝은 분위기의 팝 음악이나 라틴 음악이 적합합니다.  활기찬 리듬과 긍정적인 가사의 음악을 통해 기분 좋은 하루를 보내실 수 있습니다.


추운 날씨에는 따뜻한 느낌의 음악을, 맑은 날씨에는 밝고 경쾌한 음악을 듣는 것이 기분 전환에 도움이 됩니다.  날씨에 따라 음악을 선택하는 것은 개인의 취향에 따라 다를 수 있지만, 일반적으로 날씨와 음악의 분위기가 조화를 이루는 것이 좋습니다.  오늘 하루 즐거운 음악 감상으로 행복한 시간 보내시길 바랍니다.


------------------------------------ 
사용자: 혹시 영화추천도 가능하나요 ?
Agent Task : 영화추천


AI Agent : 네, 영화 추천 가능합니다!

1.  **추천 영화:**  2024년 개봉작 중 호평을 받은 `<악마와의 토크쇼>`와 `<퓨리오사: 매드맥스 사가>`를 추천합니다.  `<악마와의 토크쇼>`는 독특한 설정과 블랙코미디적 요소가 돋보이는 공포 영화이고, `<퓨리오사: 매드맥스 사가>`는 압도적인 액션과 스케일로 많은 기대를 받았던 작품입니다.


2.  **선정 이유:**  두 영화 모두 2024년 개봉작 중에서  평단과 관객 모두에게 좋은 평가를 받았으며, 장르적으로도 차별화되어 다양한 취향을 만족시킬 수 있기 때문입니다.  하나는 독특한 공포 영화, 다른 하나는 액션 블록버스터로 서로 다른 매력을 가지고 있습니다.


3.  최근 개봉작들을 중심으로 다양한 장르와 취향을 고려하여 추천해 드렸습니다.  더 자세한 정보는 왓챠피디아나 IMDb와 같은 영화 정보 사이트를 참고하시면 좋습니다.  선호하는 장르나 배우, 감독 등을 알려주시면 더욱 정확한 추천이 가능합니다.


------------------------------------ 
사용자: 오늘 날씨를 검색해서 오늘 영화도 추천해주세요.
Agent Task : 영화추천


AI Agent : 오늘(12월 27일) 날씨는 강추위와 함께 중부지방은 눈이 날리는 곳도 있다고 합니다.  따라서 오늘 볼 영화로는 따뜻한 감성을 느낄 수 있는 영화와, 흥미진진한 스토리로 집중할 수 있는 영화를 추천합니다.

1. **영화:**  "윤희에게" (잔잔한 감동과 아름다운 눈 풍경),  "겨울왕국" (화려한 애니메이션과 따뜻한 이야기)

2. **선정 이유:** "윤희에게"는 눈 내리는 풍경과 어울리는 감성적인 분위기로 추운 날씨에 어울리고, "겨울왕국"은 화려한 애니메이션으로 추위를 잊게 해줄 즐거움을 제공하기 때문입니다.


추운 날씨에 따뜻한 영화 감상으로 즐거운 시간 보내시길 바랍니다.


------------------------------------ 
